# Generador de imágenes calibradas

En este notebook se generan los archivos de las imágenes calibradas, tanto en .png como en archivo .fits

## Importación de módulos

[file_shortcuts](file_shortcuts.py) es el módulo en el cual se definen las funciones de ayuda para abrir y generar archivos fits de forma más rápida y cómoda.

[calibrator](calibrator.py) es el módulo en el cual se define la función que toma las imágenes Master para calibrar nuestra imagen.

In [30]:
import file_shortcuts as file
import calibrator as calib
import matplotlib.pyplot as plt
from numpy import log, float32
import os
import time

## Apertura de imagen astronómica

Se abren las imágenes a calibrar y se genera un array con las matrices de cada imagen

In [31]:
object_name = "Pallas_A802_FA"            # select the folder of the object to calibrate
n_images = 5                              # number of images to calibrate

path = f"./data/Lights/{object_name}/{object_name}_Light_"
bits_images, _ = file.file_array_generator(path, n_images)

## Apertura de imágenes Master (método AAVSO)

In [32]:
bias_aavso = file.file_opener("./master_files/AAVSO/master_Bias.fits")
dark_aavso = file.file_opener("./master_files/AAVSO/master_Dark.fits")
flat_aavso = file.file_opener("./master_files/AAVSO/master_Flat.fits")

## Apertura de imágenes Master (método MAD)

In [33]:
bias_MAD = file.file_opener("./master_files/MAD/master_Bias_MAD.fits")
dark_MAD = file.file_opener("./master_files/MAD/master_Dark_MAD.fits")
flat_MAD = file.file_opener("./master_files/MAD/master_Flat_MAD.fits")

## Calibración por la AAVSO

In [34]:
method = "AAVSO"
for i in range(n_images):
    # Image calibration 
    final_image_AAVSO = calib.image_calibrator(dark_aavso, bias_aavso, flat_aavso, bits_images[i])
    # fits file generation
    header = file.save_header(path, n_images)
    file.hdu_image_generator((final_image_AAVSO*10**4).astype(float32), header, object_name,"AAVSO",i+1) # *10^4 -> same order of magnitude as the original file
    # change file time to original file time
    source_file = path + f"{i+1:03d}.fits"
    target_file = f'./calibrated_files/{method}/{object_name}/Calibrated_{object_name}_{i+1:03d}.fits'
    source_mod_time = os.path.getmtime(source_file)
    target_access_time = os.path.getatime(target_file)
    os.utime(target_file, (target_access_time, source_mod_time))
    # png file generation
    fig, ax = plt.subplots(1, 1, figsize=(9, 6.5))
    ax.imshow(log(final_image_AAVSO), cmap='gray')
    ax.axis('off')
    ax.set_title(f'{object_name}_{i+1:03d} {method}')
    plt.savefig(f'./calibrated_images/{method}/{object_name}/{object_name}_{i+1:03d}.png')
    plt.close()

## Calibración por la MAD

In [35]:
method = "MAD"
for i in range(n_images): 
    # Image calibration
    final_image_MAD = calib.image_calibrator(dark_MAD, bias_MAD, flat_MAD, bits_images[i])
    # fits file generation
    header = file.save_header(path, n_images)
    file.hdu_image_generator((final_image_MAD*10**4).astype(float32), header, object_name,"MAD",i+1) # *10^4 -> same order of magnitude as the original file
    # change file time to original file time
    source_file = path + f"{i+1:03d}.fits"
    target_file = f'./calibrated_files/{method}/{object_name}/Calibrated_{object_name}_{i+1:03d}.fits'
    source_mod_time = os.path.getmtime(source_file)
    target_access_time = os.path.getatime(target_file)
    os.utime(target_file, (target_access_time, source_mod_time))
    # png file generation
    fig, ax = plt.subplots(1, 1, figsize=(9, 6.5))
    ax.imshow(log(final_image_MAD), cmap='gray')
    ax.axis('off')
    ax.set_title(f'{object_name}_{i+1:03d} {method}')
    plt.savefig(f'./calibrated_images/{method}/{object_name}/{object_name}_{i+1:03d}.png')
    plt.close()